# Particle Swarm Optimization

Sources:
- http://www.swarmintelligence.org/tutorials.php
- https://github.com/tisimst/pyswarm/blob/master/pyswarm/pso.py

At iteration $k$, particle $i$ has posistion $x^i_k$ defined recursively as:

$$x^i_{k+1} = x^i_k + v^i_{k+1}$$

where $v$ is its velocity defined as follows:

$$ v^i_{k+1} = w_k \cdot v^i_{k} + c_1 r_1 (p^i_k - x^i_k) + c_2 r_2 (p^g_k - x^i_k)$$

Where:

- $p^i_k$ is the particle's best known position
- $p^g_k$ is the global (all particles) best known position
- $w_k$ is an inertia parameter
- $c_1$ is the cognitive parameter
- $c_2$ is the social parameter
- $r_1, r_2$ are random numbers in $(0, 1)$

In plain English, the velocity update is defined as:

new_velocity = (inertia * previous_velocity) + (move in the direction of the personal best) + (move in the direction of the global best)

In [26]:
# import two useful libraries/modules
import numpy as np
import random

In [27]:
# -- global parameters
c1 = 1 # cognitive parameter
c2 = 2 # social parameter
w = .5 # inertia

# -- Particle Class

class Particle:
    """A Python Class for a simple particle."""
    
    def __init__(self, upper, lower, ndim):
        '''Initiate a Particle with an upper and lower bound, and a number of dimensions.'''
        # Initiate position and velocity randomly
        self.position = np.array([random.uniform(lower, upper) for _ in range(ndim)])
        self.velocity = np.array([random.uniform(-1, 1) for _ in range(ndim)])
        # These attributes are here to store the "memory" of the function
        self.personal_best_position = self.position # initial position
        self.personal_best_error = np.inf # infinity
    
    def __str__(self):
        '''what will be called when you print a Particle instance, could be useful for debugging.'''
        return("I am a particle with best position {},\ncurrent position {},\nand velocity {}.\n"
               .format(self.personal_best_position, self.position, self.velocity))       
    
    def current_error(self, function):
        '''Compute the current "fitness" value of a particle, given a function. Updates it if needed.'''
        current_fitness = function(self.position)
        if current_fitness < self.personal_best_error:
            self.personal_best_error = current_fitness
            self.personal_best_position = self.position
        return current_fitness
    
    def update_velocity(self, global_best):
        '''update a particle's velocity'''
        # two random coefficients in uniform([0, 1])
        r1 = random.random()
        r2 = random.random()
        self.velocity = (w*self.velocity + # inertia
                         c1*r1*(self.personal_best_position - self.position) + # cognitive term
                         c2*r2*(global_best - self.position)) # social term
            
    def move(self):
        '''Moves a Particle at a new iteration. Deals with out-of-bound cases (are they ok or not?)'''
        self.position = self.position + self.velocity
        # need to deal with when the particle goes out of bound...

In [28]:
# --- Solver Class
class PSO:    
    """Solver instance for Particle Swarm Optimizer. (Basically the mastermind which commands the Particles.)
    Initiated with:
    - num_particles: how many particles to use?
    - function: function to optimize.
    - n_iter: number of iterations to be performed.
    - lower, upper: lower and upper bounds for the search space (for now, a box i.e. they are the same accross all dimensions.
    - ndim: dimensionality of the search space.)"""
    
    def __init__(self, num_particles, function, n_iter, lower = -10, upper = 10, ndim = 3):
        '''Initiate the solver'''
        # create all the Particles, stored in a list.
        self.particles = [Particle(lower, upper, ndim) for _ in range(num_particles)]
        # store global best and associated error
        self.global_best = np.array([])
        self.global_best_error = np.inf # infinity
        self.function = function # function to be optimised
        self.n_iter = n_iter # num of iterations
        
    def update_best(self):
        '''Find the new best global position and update it.'''
        for particle in self.particles:
            # ask all particles for their current error. If better than current global best, update.
            if particle.current_error(self.function) < self.global_best_error:
                self.global_best = particle.position
                self.global_best_error = particle.current_error(self.function)
                
    def move_particles(self):
        '''Run one iteration of the algorithm. Update particles velocity and move them.'''
        for particle in self.particles:
            particle.update_velocity(self.global_best)
            particle.move()
    
    def __str__(self):
        '''Print best global position when calling `print(pso instance)`'''
        return """Current best position: {}
        With error: {}""".format(self.global_best, self.global_best_error)
    
    def go(self):
        '''Run the algorithm and print the result. By default, update us every 50 iterations.'''
        print("Running the PSO algorithm with {} particles, for {} iterations.\n".
              format(len(self.particles), self.n_iter))
        
        for iteration in range(self.n_iter):
            pso.update_best()
            pso.move_particles()
            if iteration % 50 == 0:
                print("Iteration number " + str(iteration))
                print("Current best error: " + str(self.global_best_error))
                print("\n")
                
        print("Found minimum at {} with value {}.".format(self.global_best, self.global_best_error))

### Tests

In [29]:
def sumsquares(x):
    return(sum([i**2 for i in x]))

In [30]:
pso = PSO(100, sumsquares, n_iter = 100)
pso.go()

Running the PSO algorithm with 100 particles, for 100 iterations.

Iteration number 0
Current best error: 5.619899086578133


Iteration number 50
Current best error: 1.7969898089191873e-12


Found minimum at [ 2.01899636e-12 -4.35193294e-12  1.16510133e-13] with value 2.3029241223003995e-23.


In [31]:
for p in pso.particles[0:10]:
    print(p)

I am a particle with best position [-9.75184831e-11  6.61503035e-11  1.75815876e-11],
current position [-1.16017622e-10  8.95958046e-11  3.11275389e-11],
and velocity [ 5.59408038e-11 -4.07808321e-11 -1.37623533e-11].

I am a particle with best position [-7.64955764e-10 -4.82993613e-10 -1.54311291e-09],
current position [4.27991163e-10 2.79270711e-10 8.52435784e-10],
and velocity [-1.43434432e-09 -9.55538073e-10 -2.86954432e-09].

I am a particle with best position [-1.43681788e-11  1.06918633e-11  1.81087704e-12],
current position [-1.12582475e-12 -2.11197967e-12  2.19176212e-13],
and velocity [ 2.49676882e-11 -1.43487084e-11  3.66028704e-13].

I am a particle with best position [ 4.37016889e-11 -1.60472420e-11  7.60520756e-12],
current position [ 3.38230078e-11 -2.96598996e-11  3.09576684e-12],
and velocity [-9.87868111e-12 -1.36126576e-11 -4.50944072e-12].

I am a particle with best position [ 3.06388629e-11 -1.31495045e-11  3.53845094e-11],
current position [-2.08074033e-11  9.7394

In [32]:
def shiftedsumsquares(x):
    return(sum([(i+2)**2 for i in x])) # minimum should be all -2

In [33]:
pso = PSO(20, shiftedsumsquares, n_iter = 100) # works well even with 20 swarms!
pso.go()

Running the PSO algorithm with 20 particles, for 100 iterations.

Iteration number 0
Current best error: 37.24166594162803


Iteration number 50
Current best error: 1.9526498191581535e-10


Found minimum at [-2. -2. -2.] with value 1.473166819228445e-20.


Trying the function $f(x) := f(x_1, x_2) = (x_1 + 2 x_2 − 3)^2 + (x_1 − 2)^2$

In [67]:
def newfunct(x):
    return (x[0] + 2*x[1] - 3)**2 + (x[0] - 2)**2

In [36]:
pso = PSO(10, newfunct, 100, ndim=2)
pso.go()

Running the PSO algorithm with 10 particles, for 100 iterations.

Iteration number 0
Current best error: 16.522265434700614


Iteration number 50
Current best error: 2.194751676599588e-11


Found minimum at [2.  0.5] with value 9.836307962347878e-22.


## Parallelizing

In [1]:
import multiprocessing

In [2]:
multiprocessing.cpu_count()

4

The `Particle` class doesn't change, it only looses the capacity to evaluate its own fitness.

In [ ]:
# -- global parameters
c1 = 1 # cognitive parameter
c2 = 2 # social parameter
w = .5 # inertia

# -- Particle Class

class Particle:
    """A Python Class for a simple particle."""
    
    def __init__(self, upper, lower, ndim):
        '''Initiate a Particle with an upper and lower bound, and a number of dimensions.'''
        # Initiate position and velocity randomly
        self.position = np.array([random.uniform(lower, upper) for _ in range(ndim)])
        self.velocity = np.array([random.uniform(-1, 1) for _ in range(ndim)])
        # These attributes are here to store the "memory" of the function
        self.personal_best_position = self.position # initial position
        self.personal_best_error = np.inf # infinity
    
    def __str__(self):
        '''what will be called when you print a Particle instance, could be useful for debugging.'''
        return("I am a particle with best position {},\ncurrent position {},\nand velocity {}.\n"
               .format(self.personal_best_position, self.position, self.velocity))       
    
    def update_velocity(self, global_best):
        '''update a particle's velocity'''
        # two random coefficients in uniform([0, 1])
        r1 = random.random()
        r2 = random.random()
        self.velocity = (w*self.velocity + # inertia
                         c1*r1*(self.personal_best_position - self.position) + # cognitive term
                         c2*r2*(global_best - self.position)) # social term
            
    def move(self):
        '''Moves a Particle at a new iteration. Deals with out-of-bound cases (are they ok or not?)'''
        self.position = self.position + self.velocity
        # need to deal with when the particle goes out of bound...

In [150]:
# --- Solver Class
class PSO_parallel:    
    """Parallel Solver instance for Particle Swarm Optimizer. (Basically the mastermind which commands the Particles.)
    Initiated with:
    - num_particles: how many particles to use?
    - function: function to optimize.
    - n_iter: number of iterations to be performed.
    - lower, upper: lower and upper bounds for the search space (for now, a box i.e. they are the same accross all dimensions.
    - ndim: dimensionality of the search space.)"""
    
    def __init__(self, num_particles, function, n_iter, lower = -10, upper = 10, ndim = 3):
        '''Initiate the solver'''
        # create all the Particles, stored in a list.
        self.particles = [Particle(lower, upper, ndim) for _ in range(num_particles)]
        self.fitnesses = np.array([])
        # store global best and associated error
        self.global_best = np.array([])
        self.global_best_error = np.inf # infinity
        self.function = function # function to be optimised
        self.n_iter = n_iter # num of iterations
        
    def get_fitnesses(self):
        with Pool(2) as p:
            fitnesses = p.apply(self.function, [[part.position for part in self.particles]])
        self.fitnesses = np.array(fitnesses)
    
    def update_particles(self):
        '''update particle best known personal position'''
        for i in range(len(self.fitnesses)):
            if self.fitnesses[i] < self.particles[i].personal_best_error:
                self.particles[i].personal_best_error = self.fitnesses[i]
                self.particles[i].personal_best_position = self.particles[i].position          
        
    def update_best(self):
        '''Find the new best global position and update it.'''
        if np.any(self.fitnesses < self.global_best_error):
            self.global_best_error = np.min(self.fitnesses)
            self.global_best = self.particles[np.argmin(self.fitnesses)].position
                      
    def move_particles(self):
        '''Run one iteration of the algorithm. Update particles velocity and move them.'''
        for particle in self.particles:
            particle.update_velocity(self.global_best)
            particle.move()
    
    def __str__(self):
        '''Print best global position when calling `print(pso instance)`'''
        return """Current best position: {}
        With error: {}""".format(self.global_best, self.global_best_error)
    
    def go(self):
        '''Run the algorithm and print the result. By default, update us every 50 iterations.'''
        print("Running the PSO algorithm in parallel with {} particles, for {} iterations.\n".
              format(len(self.particles), self.n_iter))
        
        for iteration in range(self.n_iter):
            # this happens in parallel (synchronous only rn)
            pso.get_fitnesses()
            # this doesn't
            pso.update_particles()
            pso.update_best()
            pso.move_particles()
            
            if iteration % 50 == 0:
                print("Iteration number " + str(iteration))
                print("Current best error: " + str(self.global_best_error))
                print("\n")
                
        print("Found minimum at {} with value {}.".format(self.global_best, self.global_best_error))

Let's try the parallelised code... Note that we need a function that takes a list and returns a list.


Also note that this is slower... Not worth parallelizing for very small number of particles and when the function is cheap to evaluate.

In [151]:
def newfunctpar(li):
    return [(x[0] + 2*x[1] - 3)**2 + (x[0] - 2)**2 for x in li]

In [152]:
pso = PSO_parallel(10, newfunctpar, 100, ndim=2)
pso.go()

Running the PSO algorithm with 10 particles, for 100 iterations.

Iteration number 0
Current best error: 44.04145948086628


Iteration number 50
Current best error: 1.2456344231397302e-11


Found minimum at [2.  0.5] with value 3.624670362823355e-21.
